In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import copy
import logging
import contextily as cx
import gtfs_kit as gk
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
import seaborn as sns
from rasterio.plot import show
import seaborn as sns
from sklearn.cluster import KMeans
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools.traveltime.models import rnn
from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory, busnetwork
from openbustools.drivecycle.physics import conditions, energy, vehicle

### Energy Use for KCM Network

In [ ]:
network_name = "kcm"
res_dir = Path("..","results","energy",network_name,"sensitivity","baseline")
epsg = 32148

file = open(res_dir / "trajectories_updated.pkl", "rb")
trajectories_updated = pickle.load(file)
file.close()

file = open(res_dir / "depot_locations.pkl", "rb")
depot_locations = pickle.load(file)
file.close()

file = open(res_dir / "cycles.pkl", "rb")
cycles = pickle.load(file)
file.close()

file = open(res_dir / "network_energy.pkl", "rb")
network_energy = pickle.load(file)
file.close()

file = open(res_dir / "network_charging.pkl", "rb")
network_charging = pickle.load(file)
file.close()

file = open(res_dir / "veh_status.pkl", "rb")
veh_status = pickle.load(file)
file.close()

file = open(res_dir / ".." / ".." / "network_sensitivity.pkl", "rb")
network_sensitivity = pickle.load(file)
file.close()

In [ ]:
network_sensitivity[network_sensitivity['file']=="baseline"]

In [ ]:
plot_df = network_energy.groupby("block_id").first()

# Drive cycle reported trip consumption
fig, axes = plt.subplots(1,1, figsize=(8,4))

sns.histplot(plot_df['block_consumption_kwh_mi'], kde=True, ax=axes)
axes.axvline(x=np.mean(plot_df['block_consumption_kwh_mi']), linestyle='dashed', color='red')
axes.text(x=np.mean(plot_df['block_consumption_kwh_mi']) + .1, y=145, s=f"Avg. {np.mean(plot_df['block_consumption_kwh_mi']):.3} kWh/mi")
axes.set_xlabel("Consumption (kWh/mi)")
axes.set_ylabel(f"Count ({len(plot_df):,} Total Blocks)")

fig.suptitle(f"Modeled BEB Block Consumption for Full KCM Network")
fig.tight_layout()
fig.savefig(Path("..", "plots", "kcm_block_consumption_distribution.png"))
plt.show()

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8,4))

sns.ecdfplot(plot_df['block_net_energy_kwh'], ax=axes)
axes.axvline(x=466, linestyle='dashed', color='red')
axes.text(x=486, y=0.4, s="Design Vehicle (466 kWh)")
axes.set_xlabel("Total Block Energy (kWh)")
axes.set_ylabel("Proportion of Blocks Met")

fig.suptitle(f"Modeled BEB Block Energy Needs for Full KCM Network")
fig.tight_layout()
fig.savefig(Path("..", "plots", "kcm_block_energy_distribution.png"))
plt.show()

In [ ]:
# Depot locations
block_starts_df = network_energy.groupby('block_id').first()
block_starts_df = gpd.GeoDataFrame(block_starts_df, geometry=gpd.points_from_xy([t.x for t in block_starts_df['start_loc']], [t.y for t in block_starts_df['start_loc']])).set_crs(epsg)
depot_df = gpd.GeoDataFrame(depot_locations, geometry=gpd.points_from_xy(depot_locations['depot_x'], depot_locations['depot_y'])).set_crs(epsg)

fig, axes = plt.subplots(1,1, figsize=(6,6))
block_starts_df.plot(ax=axes, column='depot_id', markersize=10, cmap='tab20')
depot_df.plot(ax=axes, marker='x', markersize=100, color="blue", linewidth=3)
cx.add_basemap(ax=axes, crs=block_starts_df.crs.to_string(), source=cx.providers.CartoDB.Positron)
axes.set_xticks([])
axes.set_yticks([])

fig.suptitle("Clustered Depot Locations")
fig.tight_layout()
plt.show()
fig.savefig(Path("..", "plots", "kcm_depot_locations.png"))

### Charging for KCM Network

In [ ]:
# Charging rate for 10/95% of blocks (unmanaged)
network_sensitivity[(network_sensitivity['metric'].isin(["Charger Power 95% (kW)","Charger Power 10% (kW)"])) & (network_sensitivity['file']=="baseline")]

In [ ]:
# Minimum charging rate to cover block energy (managed)
print(network_charging['min_charge_rate_managed'].iloc[0])

In [ ]:
# Blocks meeting pullout under baseline
print(len(network_charging[network_charging['charge_time_min'] < network_charging['t_until_pullout_min']]) / len(network_charging))

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8,5))
axes2 = plt.twinx()

sns.lineplot(veh_status, x='t_min_of_day', y='tot_veh_active', ax=axes, color=sns.color_palette()[0], label="Active")
sns.lineplot(veh_status, x='t_min_of_day', y='tot_veh_inactive', ax=axes, color=sns.color_palette()[1], label="Inactive")
axes.set_ylim(0,1500)
axes.set_xlabel("Time of Day (minutes)")
axes.set_ylabel("Number of Vehicles")
axes.legend().remove()

sns.lineplot(veh_status, x='t_min_of_day', y='tot_power', ax=axes2, color=sns.color_palette()[2], label="Power Unmanaged")
axes2.axhline(y=veh_status['tot_power'].max(), linestyle='dashed', color='red')
axes2.text(x=160, y=veh_status['tot_power'].max()-.1*veh_status['tot_power'].max(), s=f"Peak Demand: {veh_status['tot_power'].max()/1000:.1f} MW")
axes2.set_xlabel("Time of Day (minutes)")
axes2.set_ylabel("Power Demand (kW)")
axes2.legend().remove()

lines, labels = axes.get_legend_handles_labels()
lines2, labels2 = axes2.get_legend_handles_labels()
axes2.legend(lines + lines2, labels + labels2, loc='best')

fig.suptitle("Vehicle Status by Time of Day")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","kcm_veh_status.png"))

In [ ]:
# min_bins = np.arange(0, 1440, 15)
# veh_status['t_min_of_day_bin'] = np.digitize(veh_status['t_min_of_day'], min_bins) * 15
# plot_df = veh_status.groupby('t_min_of_day_bin').sum()

# fig, axes = plt.subplots(1,1, figsize=(8,4))
# axes2 = plt.twinx()

# sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_veh_arriving', ax=axes, color=sns.color_palette()[0], label="Vehicles Arriving (15min)")
# sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_veh_departing', ax=axes, color=sns.color_palette()[1], label="Vehicles Departing (15min)")
# axes.set_xlabel("Time of Day (minutes)")
# axes.set_ylabel("Number of Vehicles")
# axes.legend().remove()

# # sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_energy_arriving', ax=axes2, color=sns.color_palette()[0], linestyle='dashed', label="Power Unmanaged")
# sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_energy_departing', ax=axes2, color=sns.color_palette()[1], linestyle='dashed', label="Power Unmanaged")
# axes2.set_xlabel("Time of Day (minutes)")
# axes2.set_ylabel("Energy Needs (kWh)")
# axes2.legend().remove()

# lines, labels = axes.get_legend_handles_labels()
# # lines2, labels2 = axes2.get_legend_handles_labels()
# axes2.legend(lines, labels, loc='best')

# fig.suptitle("Vehicle Availability and Needs by Time of Day")
# fig.tight_layout()
# plt.show()
# fig.savefig(Path("..","plots","kcm_block_pullout.png"))

### Validate Block Energy w/KCM Report

In [ ]:
# Report table
block_energy_summary = network_energy.groupby('block_id').first()[['block_dist_mi', 'block_consumption_kwh_mi', 'block_net_energy_kwh']]
block_energy_summary.describe().transpose()

In [ ]:
# Load the daily summaries from KCM report
summary_data = standardfeeds.clean_parametrix("../data/bebdatafollowup/Viriciti_Energy_Reports-2023.csv")
summary_data = summary_data[summary_data['DateTime'] >= datetime(2023, 12, 1)]
summary_data['realtime_filename'] = summary_data['DateTime'].dt.strftime("%Y_%m_%d")
summary_data = summary_data.groupby(['realtime_filename','vehicle_id','metric']).agg({'value':'mean'}).reset_index().sort_values(['realtime_filename', 'vehicle_id', 'metric'])

# Load the most recent static feed
static_path = Path("..","data","kcm_static","2023_09_27")
static = gk.read_feed(static_path, dist_units='km')

# Load realtime data from all BEB vehicle IDs in the KCM report
realtime_path = Path("..","data","kcm_realtime","processed", "analysis")
beb_ids = summary_data['vehicle_id'].unique()
beb_dates = summary_data['realtime_filename'].unique()
all_realtime_data = []
for d in beb_dates:
    realtime_data = pd.read_pickle(Path(f"../data/kcm_realtime/processed/analysis/{d}.pkl"))
    realtime_data = realtime_data[realtime_data['vehicle_id'].isin(beb_ids)]
    realtime_data['realtime_filename'] = realtime_data['realtime_filename'].str[:-4]
    all_realtime_data.append(realtime_data)
all_realtime_data = pd.concat(all_realtime_data).sort_values(['realtime_filename','vehicle_id','trip_id','locationtime'])

# Map (day, vehicle_id) > trip_ids using the realtime data
trip_id_lookup = all_realtime_data[['realtime_filename','vehicle_id','trip_id']].drop_duplicates().copy()
# Map trip_id > (service_id, block_id) using the static data
block_id_lookup = static.get_trips()[['service_id','block_id','trip_id']].drop_duplicates().copy()
block_id_lookup = pd.merge(trip_id_lookup, block_id_lookup, on='trip_id')
block_id_lookup = block_id_lookup[['realtime_filename','vehicle_id','service_id','block_id']].drop_duplicates().copy()
# Join energy summaries to their block_ids; note there are days where the vehicle was tracked on multiple blocks in the realtime
summary_data = pd.merge(summary_data, block_id_lookup, on=['realtime_filename','vehicle_id'])
# Get comparison metrics for each block
summary_data_means = summary_data.groupby(['block_id','metric'], as_index=False).agg({'value': 'mean'}).pivot(index='block_id', columns='metric', values='value')
summary_data_stds = summary_data.groupby(['block_id','metric'], as_index=False).agg({'value': 'std'}).pivot(index='block_id', columns='metric', values='value').mean()
real_beb_comparison = pd.merge(summary_data_means, block_energy_summary, on='block_id')
real_beb_comparison.head()

In [ ]:
plot_df = real_beb_comparison
plot_df['block_dist_60ft_mi'] = plot_df['block_dist_mi'] * 1.6
plot_df['block_consumption_60ft_kwh_mi'] = plot_df['block_consumption_kwh_mi'] * 1.5
plot_df['block_net_energy_60ft_kwh'] = plot_df['block_consumption_60ft_kwh_mi'] * plot_df['block_dist_60ft_mi']

fig, axes = plt.subplots(1,2, figsize=(8,4))
sns.scatterplot(data=plot_df, x='block_net_energy_kwh', y='Energy used', ax=axes[0])
axes[0].axline([0,0], [1,1], color='red', linestyle='--', alpha=.8)
axes[0].axline([0, 0+summary_data_stds['Energy used']], [1, 1+summary_data_stds['Energy used']], color='red', alpha=.3)
axes[0].axline([0, 0-summary_data_stds['Energy used']], [1, 1-summary_data_stds['Energy used']], color='red', alpha=.3)
axes[0].set_xlim(0, 800)
axes[0].set_ylim(0, 800)
axes[0].set_xlabel("Modeled Energy (kWh)")
axes[0].set_ylabel("Actual Energy (kWh)")
axes[0].set_title("40ft Bus")

sns.scatterplot(data=plot_df, x='block_net_energy_60ft_kwh', y='Energy used', ax=axes[1])
axes[1].axline([0,0], [1,1], color='red', linestyle='--', alpha=.8)
axes[1].axline([0, 0+summary_data_stds['Energy used']], [1, 1+summary_data_stds['Energy used']], color='red', alpha=.3)
axes[1].axline([0, 0-summary_data_stds['Energy used']], [1, 1-summary_data_stds['Energy used']], color='red', alpha=.3)
axes[1].set_xlim(0, 800)
axes[1].set_ylim(0, 800)
axes[1].set_xlabel("Modeled Energy (kWh)")
axes[1].set_ylabel("Actual Energy (kWh)")
axes[1].set_title("Adjusted for 60ft Bus")
fig.suptitle(f"Modeled vs. Actual Block Energy Consumption for KCM\n{len(plot_df)} Blocks")
fig.tight_layout()
fig.savefig(Path("..", "plots", "kcm_real_beb_comparison.png"))
plt.show()

### KCM Sensitivity Analysis and Performance Metrics

In [ ]:
baseline_sensitivity = network_sensitivity[network_sensitivity['file']=="baseline"]

In [ ]:
plot_metrics = [
    'Avg. Block Consumption (kWh/mi)',
    'Avg. Trip Consumption (kWh/mi)',
]
plot_parameters = [
    'Acc./Dec. Factor',
    "Aux Power",
    "Deadhead Consumption",
    "Depot Density",
    "Door Open Time",
    "Passenger Load",
    "Temperature"
]

plot_df = network_sensitivity[network_sensitivity['metric'].isin(plot_metrics)].sort_values(['metric', 'file'])
plot_df = plot_df[plot_df['sensitivity_parameter'].isin(plot_parameters)]

# Compare metrics across sensitivity parameters
fig, axes = plt.subplots(2, 1, figsize=(8,5))
axes = axes.flatten()
for i, metric in enumerate(plot_df['metric'].unique()):
    sns.boxplot(plot_df[plot_df['metric']==metric], x='value', y='sensitivity_parameter', ax=axes[i])
    axes[i].vlines(x=baseline_sensitivity[baseline_sensitivity['metric']==metric]['value'].iloc[0], ymin=-1, ymax=len(plot_parameters), color='red', linestyle='--')
    axes[i].set_ylabel("")
    axes[i].set_xlabel(metric)
    axes[i].set_xlim(0.5, 6.0)

fig.suptitle("Sensitivity of Energy Consumption in Full KCM Network")
fig.tight_layout()
plt.show()
fig.savefig(Path("..", "plots", "kcm_sensitivity_consumption.png"))

In [ ]:
plot_metrics = [
    'Avg. Block Energy (kWh)',
    'Battery Capacity 10% (kWh)',
    'Battery Capacity 95% (kWh)'
]
plot_parameters = [
    'Acc./Dec. Factor',
    "Aux Power",
    "Deadhead Consumption",
    "Depot Density",
    "Door Open Time",
    "Passenger Load",
    "Temperature"
]

plot_df = network_sensitivity[network_sensitivity['metric'].isin(plot_metrics)].sort_values(['metric', 'file'])
plot_df = plot_df[plot_df['sensitivity_parameter'].isin(plot_parameters)]
plot_df

# Compare metrics across sensitivity parameters
fig, axes = plt.subplots(3, 1, figsize=(8,7.5))
axes = axes.flatten()
for i, metric in enumerate(plot_df['metric'].unique()):
    sns.boxplot(plot_df[plot_df['metric']==metric], x='value', y='sensitivity_parameter', ax=axes[i])
    axes[i].vlines(x=baseline_sensitivity[baseline_sensitivity['metric']==metric]['value'].iloc[0], ymin=-1, ymax=len(plot_parameters), color='red', linestyle='--')
    axes[i].set_ylabel("")
    axes[i].set_xlabel(metric)

fig.suptitle("Sensitivity of Block Energy Needs in Full KCM Network")
fig.tight_layout()
plt.show()
fig.savefig(Path("..", "plots", "kcm_sensitivity_energy.png"))

In [ ]:
# TODO: How to show the managed vs unmanaged charging results for KCM; set up multicity?
# TODO: Why do some networks not work?

# Low costs essential; no managed charging, depot only, only some blocks (more consistent energy use too)
# Every fleet could do an initial rollout; but to reach full fleet the technology simply needs to improve (+/- sensitivity values on either side)
# The absolute cutting edge of technology is not or is needed to reach full fleet electrification. Are full 2030 fleet electrification plans feasible? Coming back to cost; will it save money? Accomplish emissions goals?
# Haven't tested charging strategies and more; but initial rollouts are feasible without any management at all
# Enthusiastic about electrification plans, but need to be realistic about the challenges and costs incurred by agencies already under pressure
# Bring back 44% cannot see path forwards to electrification
# To meet neeeds of 95% unmanaged charging, the plug power must be high which strongly affects peak loads

In [ ]:
# plot_metrics = [
#     'Charger Power 10% (kW)',
#     'Charger Power 95% (kW)',
# ]
# plot_parameters = [
#     'Acc./Dec. Factor',
#     "Aux Power",
#     "Deadhead Consumption",
#     "Depot Density",
#     "Door Open Time",
#     "Passenger Load",
#     "Temperature",
# ]

# plot_df = network_sensitivity[network_sensitivity['metric'].isin(plot_metrics)].sort_values(['metric', 'file'])
# plot_df = plot_df[plot_df['sensitivity_parameter'].isin(plot_parameters)]
# plot_df

# # Compare metrics across sensitivity parameters
# fig, axes = plt.subplots(2, 1, figsize=(8,5))
# axes = axes.flatten()
# for i, metric in enumerate(plot_df['metric'].unique()):
#     sns.boxplot(plot_df[plot_df['metric']==metric], x='value', y='sensitivity_parameter', ax=axes[i])
#     axes[i].vlines(x=baseline_sensitivity[baseline_sensitivity['metric']==metric]['value'].iloc[0], ymin=-1, ymax=len(plot_parameters), color='red', linestyle='--')
#     axes[i].set_ylabel("")
#     axes[i].set_xlabel(metric)

# fig.suptitle("Sensitivity of Unmanaged Charging Needs")
# fig.tight_layout()
# plt.show()
# fig.savefig(Path("..", "plots", "kcm_sensitivity_charging.png"))

In [ ]:
# plot_metrics = [
#     'Min Charge Rate (kW)',
# ]
# plot_parameters = [
#     'Acc./Dec. Factor',
#     "Aux Power",
#     "Deadhead Consumption",
#     "Depot Density",
#     "Door Open Time",
#     "Passenger Load",
#     "Temperature",
# ]

# plot_df = network_sensitivity[network_sensitivity['metric'].isin(plot_metrics)].sort_values(['metric', 'file'])
# plot_df = plot_df[plot_df['sensitivity_parameter'].isin(plot_parameters)]
# plot_df

# # Compare metrics across sensitivity parameters
# fig, axes = plt.subplots(1, 1, figsize=(10,4))
# # axes = axes.flatten()
# for i, metric in enumerate(plot_df['metric'].unique()):
#     sns.boxplot(plot_df[plot_df['metric']==metric], x='value', y='sensitivity_parameter', ax=axes)
#     axes.vlines(x=baseline_sensitivity[baseline_sensitivity['metric']==metric]['value'].iloc[0], ymin=-1, ymax=len(plot_parameters), color='red', linestyle='--')
#     axes.set_ylabel("")
#     axes.set_xlabel(metric)

# fig.suptitle("Sensitivity of 'Managed' Charging")
# fig.tight_layout()
# plt.show()
# fig.savefig(Path("..", "plots", "kcm_sensitivity_charging.png"))

In [ ]:
# plot_metrics = [
#     'Peak 15min Power (kW)',
#     'Avg. 15min Power (kW)',
#     'Avg. Charge Time (min)'
# ]
# plot_parameters = [
#     'Acc./Dec. Factor',
#     "Aux Power",
#     "Deadhead Consumption",
#     "Depot Density",
#     "Door Open Time",
#     "Passenger Load",
#     "Temperature",
#     "Plug Power"
# ]

# plot_df = network_sensitivity[network_sensitivity['metric'].isin(plot_metrics)].sort_values(['metric', 'file'])
# plot_df = plot_df[plot_df['sensitivity_parameter'].isin(plot_parameters)]
# plot_df

# # Compare metrics across sensitivity parameters
# fig, axes = plt.subplots(3, 1, figsize=(10,12))
# axes = axes.flatten()
# for i, metric in enumerate(plot_df['metric'].unique()):
#     sns.boxplot(plot_df[plot_df['metric']==metric], x='value', y='sensitivity_parameter', ax=axes[i])
#     axes[i].vlines(x=baseline_sensitivity[baseline_sensitivity['metric']==metric]['value'].iloc[0], ymin=-1, ymax=len(plot_parameters), color='red', linestyle='--')
#     axes[i].set_ylabel("")
#     axes[i].set_xlabel(metric)

# fig.suptitle("Sensitivity of Unmanaged Power Demand")
# fig.tight_layout()
# plt.show()
# fig.savefig(Path("..", "plots", "kcm_unmanaged_power_sensitivity.png"))

### All Networks Sensitivity Analysis and Performance Metrics

In [ ]:
# cleaned_sources = pd.read_csv("../data/cleaned_sources.csv")

# all_network_res = []
# for i,row in cleaned_sources.iterrows():
#     try:
#         network_sensitivity = pd.read_pickle(Path("..","results","energy",row['uuid'],"network_sensitivity.pkl"))
#         all_network_res.append(network_sensitivity)
#     except Exception as e:
#         continue
# all_network_res = pd.concat(all_network_res)

In [ ]:
# print(all_network_res['metric'].unique())
# print(all_network_res['sensitivity_parameter'].unique())

In [ ]:
# plot_df = all_network_res[all_network_res['metric'].isin(["Battery Capacity 95% (kWh)"])]
# plot_df = plot_df[plot_df['sensitivity_parameter']=="Aux Power"]
# plot_df

# fig, axes = plt.subplots(1, 1, figsize=(12,6))

# for f in plot_df['file'].unique():
#     sns.ecdfplot(plot_df[plot_df['file']==f]['value'], label=f)

# axes.set_xlabel("Battery Capacity Needed for Full Electrification")
# axes.set_ylabel("Proportion of Networks")
# axes.legend()

In [ ]:
# plot_df = all_network_res[all_network_res['metric'].isin(["Battery Capacity 95% (kWh)"])]
# plot_df = plot_df[plot_df['sensitivity_parameter']=="Aux Power"]
# plot_df

# fig, axes = plt.subplots(1, 1, figsize=(12,6))

# for f in plot_df['file'].unique():
#     sns.ecdfplot(plot_df[plot_df['file']==f]['value'], label=f)

# axes.set_xlabel("Battery Capacity Needed for Full Electrification")
# axes.set_ylabel("Proportion of Networks")
# axes.legend()

In [ ]:
# plot_df = all_network_res[all_network_res['metric'].isin(["Battery Capacity 10% (kWh)"])]
# plot_df = plot_df[plot_df['sensitivity_parameter']=="Aux Power"]
# plot_df

# fig, axes = plt.subplots(1, 1, figsize=(12,6))

# for f in plot_df['file'].unique():
#     sns.ecdfplot(plot_df[plot_df['file']==f]['value'], label=f)

# axes.set_xlabel("Battery Capacity Needed for Unmanaged Depot Charging Pilot")
# axes.set_ylabel("Proportion of Networks")
# axes.legend()

In [ ]:
# plot_df_1 = all_network_res[all_network_res['metric']=="Min Charge Rate (kW)"]
# plot_df_2 = all_network_res[all_network_res['metric']=="Avg. Block Duration (min)"]
# plot_df = pd.merge(plot_df_1, plot_df_2, on=['file','provider'])
# plot_df = plot_df[plot_df['file']=="aux_power_kw-2"]
# plot_df.head()

# fig, axes = plt.subplots(1,1,figsize=(15,8))

# sns.scatterplot(x=plot_df['value_x'], y=plot_df['value_y'], hue=plot_df['provider'], ax=axes)
# axes.set_xlabel("Minimum Charge Rate (kW)")
# axes.set_ylabel("Avg. Block Duration (min)")

# fig.suptitle("Block Duration vs. Minimum Charge Rate")